In [2]:
from flask import Flask, request, jsonify, render_template, session, send_file, redirect, url_for
from flask_session import Session  # You might need to install flask-session
from openpyxl import load_workbook
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
from io import BytesIO  # Import BytesIO for in-memory binary streams
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

app = Flask(__name__)
# Configure the Flask app to use server-side session
app.config["SESSION_PERMANENT"] = False
app.config["SESSION_TYPE"] = "filesystem"
Session(app)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///bids.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)


class Bid(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(120), unique=True, nullable=False)
    # You can add more fields here as needed, such as bid date, total cost, etc.

    def __repr__(self):
        return f'<Bid {self.name}>'


@app.route('/bid-management', methods=['GET', 'POST'])
def bid_management():
    if request.method == 'POST':
        bid_name = request.form.get('bidName')
        create_new_bid = request.form.get('createNewBid') == 'on'

        if create_new_bid:
            existing_bid = Bid.query.filter_by(name=bid_name).first()
            if existing_bid is None:
                new_bid = Bid(name=bid_name)
                db.session.add(new_bid)
                db.session.commit()
                return redirect(url_for('bid_management', bid_id=new_bid.id))
            else:
                return "A bid with this name already exists.", 400
        else:
            selected_bid = Bid.query.filter_by(name=bid_name).first()
            if selected_bid:
                return redirect(url_for('bid_management', bid_id=selected_bid.id))
            else:
                return "Bid not found.", 404

    # For GET request or initial page load
    bids = Bid.query.all()
    return render_template('BidManagement.html', bids=bids)


def load_sheets(filename):
    workbook = load_workbook(filename)
    inventory = []
    factors = []

    # Load Inventory Sheet
    inventory_sheet = workbook["Inventory"]
    # Correctly accessing headers as values directly, without trying to access .value on them
    inventory_headers = [cell for cell in next(inventory_sheet.iter_rows(min_row=1, max_row=1, values_only=True))]
    for row in inventory_sheet.iter_rows(min_row=2, values_only=True):
        item = {inventory_headers[i]: value for i, value in enumerate(row)}
        inventory.append(item)
    
    # Load Factor Sheet (assuming similar structure, ensure it's adjusted accordingly)
    factor_sheet = workbook["Factor"]
    factor_headers = [cell for cell in next(factor_sheet.iter_rows(min_row=1, max_row=1, values_only=True))]
    for row in factor_sheet.iter_rows(min_row=2, values_only=True):
        item = {factor_headers[i]: value for i, value in enumerate(row)}
        factors.append(item)
    
    return inventory, factors

inventory, factors = load_sheets('Input.xlsx')

@app.route('/inventory/manage', methods=['GET', 'POST'])
def manage_inventory():
    # Check if inventory data is already loaded into the session
    if 'loaded_inventory' not in session:
        # Load inventory and factors from Excel sheets
        inventory, factors = load_sheets('Input.xlsx')
        # Store only the inventory in the session for use in the template
        session['inventory'] = inventory
        # Set a flag to indicate inventory data has been loaded to avoid reloading it on subsequent requests
        session['loaded_inventory'] = True
    session.setdefault('inventory', [])  

    if request.method == 'POST':
        part_num = request.form.get('PartNum')
        description = request.form.get('Description')
        cost = request.form.get('Cost')
        factor_code = request.form.get('FactorCode')  # Capture Factor Code from the form

        existing_item = next((item for item in session['inventory'] if item['PartNum'] == part_num), None)
        if existing_item:
            existing_item['Description'] = description
            existing_item['Cost'] = cost
            existing_item['FactorCode'] = factor_code  # Ensure this line is correctly updating the Factor Code
        else:
            session['inventory'].append({
                'PartNum': part_num,
                'Description': description,
                'Cost': cost,
                'FactorCode': factor_code  # Include Factor Code for new items
            })
        session.modified = True
        return redirect(url_for('manage_inventory'))

    else:
        # Standardize the "FactorCode" key in inventory items before rendering
        standardized_inventory = []
        for item in session.get('inventory', []):
            # Ensure consistency in key for "Factor Code"
            if 'Factor Code' in item:
                # If 'Factor Code' exists, move its value to 'FactorCode' and remove the 'Factor Code' key
                item['FactorCode'] = item.pop('Factor Code')
            standardized_inventory.append(item)

        # Use standardized_inventory for rendering, ensuring consistency in handling "Factor Codes"
        print(standardized_inventory)  # Optional: Print to verify the standardization
        return render_template('ManageInventory.html', inventory=standardized_inventory)



@app.route('/inventory/delete', methods=['POST'])
def delete_inventory_item():
    part_num = request.form.get('PartNum')
    session['inventory'] = [item for item in session.get('inventory', []) if item['PartNum'] != part_num]
    session.modified = True  # Mark the session as modified to save changes
    return redirect(url_for('manage_inventory'))

@app.route('/new-bid')
def new_bid():
    session.setdefault('bid_items', [])
    total_cost = sum(item['Cost'] * item['Quantity'] for item in session['bid_items'])
    return render_template('Bid_Job_Estimating.html', bid_items=session['bid_items'], total_cost=total_cost)


@app.route('/')
def home():
    # Basic homepage with links to other parts of the application
    return render_template('Home.html')

@app.route('/add', methods=['POST'])
def add_item():
    part_num = request.form['PartNum']
    quantity = int(request.form['Quantity'])
    item = next((item for item in inventory if item['PartNum'] == part_num), None)
    if item:
        # Append to bid_items in session
        session['bid_items'].append({**item, "Quantity": quantity, "Cost": float(item['Cost'])})
        session.modified = True  # Mark the session as modified to save changes
        total_cost = sum(item['Cost'] * item['Quantity'] for item in session['bid_items'])
        return jsonify(success=True, bid_items=session['bid_items'], total_cost=total_cost)
    else:
        return jsonify(success=False)

@app.route('/factors/manage', methods=['GET', 'POST'])
def manage_factors():
    if request.method == 'POST':
        factor_id = request.form.get('Factor_ID')
        description = request.form.get('Description')
        labor_hours = request.form.get('LaborHours')

        # Load factors from the Excel file on POST request as well to ensure list is up-to-date
        _, factors = load_sheets('Input.xlsx')

        # Attempt to find an existing factor by ID
        existing_factor = next((factor for factor in factors if factor['Factor_ID'] == factor_id), None)

        if existing_factor:
            # Update existing factor
            existing_factor['Description'] = description
            existing_factor['Labor Hours'] = float(labor_hours)
        else:
            # Add new factor
            factors.append({
                'Factor_ID': factor_id,
                'Description': description,
                'Labor Hours': float(labor_hours)
            })

        # Optionally, save updated factors list back to the Excel file here

        return redirect(url_for('manage_factors'))
    else:
        # Load factors from the Excel file on each GET request
        _, factors = load_sheets('Input.xlsx')
        return render_template('ManageFactors.html', factors=factors)



@app.route('/factors/delete', methods=['POST'])
def delete_factor_code():
    factor_id = request.form.get('Factor_ID')
    session['factors'] = [factor for factor in session.get('factors', []) if factor['Factor_ID'] != factor_id]
    session.modified = True  # Mark the session as modified to save changes
    return redirect(url_for('manage_factors'))


@app.route('/delete', methods=['POST'])
def delete_items():
    selected_items = request.json['selectedItems']
    # Assuming 'bid_items' is stored in session and is a list of dictionaries
    session['bid_items'] = [item for item in session.get('bid_items', []) if item['PartNum'] not in selected_items]
    total_cost = sum(item['Cost'] * item['Quantity'] for item in session['bid_items'])
    return jsonify({'bid_items': session['bid_items'], 'total_cost': total_cost})

@app.route('/inventory')
def get_inventory():
    return jsonify(inventory)

@app.route('/factors')
def get_factors():
    # Assuming factors is a list of dictionaries
    try:
        # Debug print to inspect the structure
        print(factors)
        return jsonify(factors)
    except TypeError as e:
        # Log the error for debugging
        print(f"Error serializing factors: {e}")
        # Return an error message or empty list as a fallback
        return jsonify([]), 500

@app.route('/add-update-bid', methods=['GET', 'POST'])
def add_update_bid():
    if request.method == 'POST':
        create_new_bid = request.form.get('CreateNewBid') == 'on'
        bid_name = request.form.get('BidName')
        
        if create_new_bid:
            # Check if a bid with the same name already exists
            existing_bid = Bid.query.filter_by(name=bid_name).first()
            if existing_bid is None:
                # Create new bid and add it to the database
                new_bid = Bid(name=bid_name)
                db.session.add(new_bid)
                db.session.commit()
                # Assuming you have a way to pass the created bid's ID or name to the new bid page
                session['current_bid_id'] = new_bid.id
                return redirect(url_for('new_bid'))
            else:
                # Handle the case where a bid with the same name already exists
                return "A bid with this name already exists.", 400
        else:
            # Handle updating an existing bid, if applicable
            pass
    else:
        return render_template('AddUpdateBid.html')
@app.route('/create-proposal')
def create_proposal():
    next_tab = request.args.get('next_tab', None)
    return render_template('create_proposal.html', next_tab=next_tab)
 

@app.route('/create-proposal_report')
def create_proposal_report():
    proposal_data = session.get('proposal_data', {})
    buffer = BytesIO()
    c = canvas.Canvas(buffer, pagesize=letter)
    # Adjust y as needed
    y = 750
    
    # Use proposal_data to draw text on the PDF
    c.drawString(100, y, f"Bid ID: {proposal_data.get('bidID', 'N/A')}")
    # Continue for other fields
    
    c.save()
    buffer.seek(0)
    return send_file(buffer, as_attachment=True, download_name='proposal.pdf', mimetype='application/pdf')



@app.route('/submit-proposal-data', methods=['POST'])
def submit_proposal_data():
    # Store or update proposal data in session
    session['proposal_data'] = request.form.to_dict()
    
    # Extract the next tab from the form data
    next_tab = session['proposal_data'].get('next_tab', 'defaultTabName')
    
    session.modified = True  # Ensure the session is marked as modified
    
    # Redirect back to the form page and include the next_tab as a query parameter
    return redirect(url_for('create_proposal', next_tab=next_tab))


@app.route('/insert', methods=['POST'])
def insert_item():
    insert_after = request.form['InsertAfter']
    part_num = request.form['PartNum']
    quantity = int(request.form['Quantity'])
    item = next((item for item in inventory if item['PartNum'] == part_num), None)
    if item:
        index = next((i for i, item in enumerate(session['bid_items']) if item['PartNum'] == insert_after), None)
        if index is not None:
            session['bid_items'].insert(index + 1, {**item, "Quantity": quantity, "Cost": float(item['Cost'])})
            session.modified = True  # Mark the session as modified to save changes
            total_cost = sum(item['Cost'] * item['Quantity'] for item in session['bid_items'])
            return jsonify(success=True, bid_items=session['bid_items'], total_cost=total_cost)
    return jsonify(success=False)

with app.app_context():
    db.create_all()

if __name__ == '__main__':
    app.run(port=5000, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [21/Mar/2024 15:06:40] "GET /bidmanagement HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2024 15:06:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2024 15:06:43] "GET /bid-management HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2024 15:06:49] "POST /bid-management HTTP/1.1" 302 -
127.0.0.1 - - [21/Mar/2024 15:06:49] "GET /bid-management?bid_id=1 HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2024 15:06:54] "POST /bid-management?bid_id=1 HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2024 15:06:58] "POST /bid-management?bid_id=1 HTTP/1.1" 302 -
127.0.0.1 - - [21/Mar/2024 15:06:58] "GET /bid-management?bid_id=2 HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2024 15:07:05] "POST /bid-management?bid_id=2 HTTP/1.1" 302 -
127.0.0.1 - - [21/Mar/2024 15:07:05] "GET /bid-management?bid_id=1 HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2024 15:07:09] "POST /bid-management?bid_id=1 HTTP/1.1" 302 -
127.0.0.1 - - [21/Mar/2024 15:07:09] "GET /bid-management?bid_id=1 HTTP/1.1"